In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the Dataset class and helper functions in dataset.py
%%writefile dataset.py
from glob import glob
from os import path
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset

class ImagesDataset(Dataset):
    def __init__(self, image_dir, labels_file, width=100, height=100, dtype=None):
        self.image_filepaths = sorted(path.abspath(f) for f in glob(path.join(image_dir, "*.jpg")))
        self.filenames_classnames, self.classnames_to_ids = ImagesDataset.load_classnames(labels_file)
        if width < 100 or height < 100:
            raise ValueError('width and height must be greater than or equal 100')
        self.width = width
        self.height = height
        self.dtype = dtype

    @staticmethod
    def load_classnames(labels_file):
        filenames_classnames = np.genfromtxt(labels_file, delimiter=';', skip_header=1, dtype=str)
        classnames = np.unique(filenames_classnames[:, 1])
        classnames.sort()
        classnames_to_ids = {classname: index for index, classname in enumerate(classnames)}
        return filenames_classnames, classnames_to_ids

    def __getitem__(self, index):
        with Image.open(self.image_filepaths[index]) as im:
            image = np.array(im, dtype=self.dtype)
        image = to_grayscale(image)
        resized_image, _ = prepare_image(image, self.width, self.height, 0, 0, 32)
        resized_image = torch.tensor(resized_image, dtype=torch.float32) / 255.0
        classname = self.filenames_classnames[index][1]
        classid = self.classnames_to_ids[classname]
        return resized_image, classid, classname, self.image_filepaths[index]

    def __len__(self):
        return len(self.image_filepaths)

def to_grayscale(pil_image):
    if pil_image.ndim == 2:
        return pil_image.copy()[None]
    if pil_image.ndim != 3:
        raise ValueError("image must have either shape (H, W) or (H, W, 3)")
    if pil_image.shape[2] != 3:
        raise ValueError(f"image has shape (H, W, {pil_image.shape[2]}), but it should have (H, W, 3)")

    rgb = pil_image / 255
    rgb_linear = np.where(
        rgb < 0.04045,
        rgb / 12.92,
        ((rgb + 0.055) / 1.055) ** 2.4
    )
    grayscale_linear = 0.2126 * rgb_linear[..., 0] + 0.7152 * rgb_linear[..., 1] + 0.0722 * rgb_linear[..., 2]

    grayscale = np.where(
        grayscale_linear < 0.0031308,
        12.92 * grayscale_linear,
        1.055 * grayscale_linear ** (1 / 2.4) - 0.055
    )
    grayscale = grayscale * 255

    if np.issubdtype(pil_image.dtype, np.integer):
        grayscale = np.round(grayscale)
    return grayscale.astype(pil_image.dtype)[None]

def prepare_image(image, width, height, x, y, size):
    if image.ndim < 3 or image.shape[-3] != 1:
        raise ValueError("image must have shape (1, H, W)")
    if width < 32 or height < 32 or size < 32:
        raise ValueError("width/height/size must be >= 32")
    if x < 0 or (x + size) > width:
        raise ValueError(f"x={x} and size={size} do not fit into the resized image width={width}")
    if y < 0 or (y + size) > height:
        raise ValueError(f"y={y} and size={size} do not fit into the resized image height={height}")

    image = image.copy()

    if image.shape[1] > height:
        image = image[:, (image.shape[1] - height) // 2: (image.shape[1] - height) // 2 + height, :]
    else:
        image = np.pad(image, ((0, 0), ((height - image.shape[1])//2, np.ceil((height - image.shape[1])/2)), (0, 0)), mode='edge')

    if image.shape[2] > width:
        image = image[:, :, (image.shape[2] - width) // 2: (image.shape[2] - width) // 2 + width]
    else:
        image = np.pad(image, ((0, 0), (0, 0), ((width - image.shape[2])//2, np.ceil((width - image.shape[2])/2))), mode='edge')

    subarea = image[:, y:y + size, x:x + size]
    return image, subarea

Overwriting dataset.py


In [ ]:
# Define the Model Architecture in architecture.py
%%writefile architecture.py
import torch
import torch.nn as nn

class MyCNN(nn.Module):
    def __init__(self, num_classes=20):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 12 * 12, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 12 * 12)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyCNN(num_classes=20)

Overwriting architecture.py


In [ ]:
# Train the Model and Save the Trained Model
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset import ImagesDataset
from architecture import MyCNN, model

# Define data directory and load dataset
data_dir = '/content/drive/My Drive/training_data'

train_dataset = ImagesDataset(image_dir=data_dir, dtype=np.uint8)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
best_loss = float('inf')

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels, _, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), "model.pth")

print('Finished Training')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/10], Loss: 2.4780
Epoch [2/10], Loss: 1.9194
Epoch [3/10], Loss: 1.6156
Epoch [4/10], Loss: 1.3288
Epoch [5/10], Loss: 1.0322
Epoch [6/10], Loss: 0.7235
Epoch [7/10], Loss: 0.4665
Epoch [8/10], Loss: 0.3007
Epoch [9/10], Loss: 0.1979
Epoch [10/10], Loss: 0.1537
Finished Training


In [ ]:
# Save the Trained Model to Google Drive
!cp model.pth /content/drive/My\ Drive/model.pth

In [ ]:
# Load the Trained Model
model = MyCNN(num_classes=20)
model.load_state_dict(torch.load('/content/drive/My Drive/model.pth'))
model.eval()

MyCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=18432, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=20, bias=True)
)

In [ ]:
# Define a Function to Make Predictions on New Images
from PIL import Image
import torchvision.transforms as transforms

def predict_image(image_path, model, classnames):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((100, 100)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        classname = classnames[predicted.item()]
    return classname

In [ ]:
# Test the Model with Various Images
classnames = list(train_dataset.classnames_to_ids.keys())
test_image_path = '/content/drive/My Drive/training_data/0001661.jpg'  # Replace with the path to your test image
predicted_class = predict_image(test_image_path, model, classnames)
print(f'The predicted class for the test image is: {predicted_class}')

The predicted class for the test image is: tree
